En primer lugar vamos a importar las librerias necesarias

In [15]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import PlaintextCorpusReader
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import pyLDAvis.gensim
import nltk, re, pprint, spacy


Ahora creamos el conjunto de documentos. Cada documento se va a crear como una string, de forma que el mismo sistema nos sirviese si tomamos documentos como ficheros de un directorio. Con ellos constgruiremos el corpus de documentos, que será una lista de strings

In [16]:
doc_list=[]
docs_dir = "/Users/gandalf/Desktop/Tokens"
documentos = PlaintextCorpusReader(docs_dir, '.*txt')

for fileid in documentos.fileids():
    doc_list.append(documentos.raw(fileids=fileid))

# print(doc_list[:1]) 
    

Vamos a cargar el modelo de español para spaCy y vamos a cargar la lista de palabras vacías en español

In [17]:
# Cargamos el modelo en español para dividir el texto y obtener formas normales (stemms)

nlp = spacy.load('es_core_news_sm')

# Y cargamos una lista de palabras vacías en español

es_stop = get_stop_words('es')

Y ahora, a partir de los documentos en formato lista de strings, vamos a obtener el corpus en el formato que neecesitamos.
Vamos a crear el corpus como una lista. 
Dentro de esa lista, cada documento será una lista de strings. 
Cada string será un término del documento.
Para que los términos no nos distorsionen el análisis LDA, vamos a poner todos los términos en minúsculas, 
pasamos a formas normales y eliminamos palabras vacías

In [6]:
# Lista que incorporará el corpus completo

textos = []
palabras = []

# Y ahora un bucle que pasa por cada documento
for doc in doc_list:
    
    # Pasamos el documento a minusculas
    doc_min = doc.lower()
    
    # Dividimos en palabras y sacamos formas normales con spaCy
    doc = nlp(doc_min)
    
    # Y ahora añadimos la palabra, pero en su forma normal. Sólo añadimos la palabra si no es una palabra vacía.
    # Además, si la palabra sólo tiene 1 caracter, no la añado. Probablemente sea un signo de puntuación o una palabra vacía
    
    for token in doc:
        if (not token.lemma_ in es_stop) and (len(token.lemma_)>1):
            palabras.append(token.lemma_)
    
    # Y por último añadimos a los textos el documento ya tratado (formas normales, no palabras vacías...)
    
    textos.append(palabras)
    palabras = []



In [7]:
# print(doc_list)

In [8]:
# print(textos)

In [9]:
# Construímos el diccionario, que es una lista de las palabras que aparecen en el corpus textos

dictionary = corpora.Dictionary(textos)
    
# Ahora generamos la matriz términos documentos que nos hace falta para llamar a la función que genera el modelo LDA
corpus = [dictionary.doc2bow(texto) for texto in textos]

# Y por último generamos el modelo LDA
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=7, id2word = dictionary, passes=50)

Y ahora vamos a imprimir los resultados. 

In [10]:
print(ldamodel.print_topics(num_topics=7, num_words=10))

[(0, '0.015*"\n\n" + 0.013*"ser" + 0.011*"parir" + 0.010*"consumidor" + 0.010*"leche" + 0.009*"querer" + 0.008*"pagar" + 0.007*"preciar" + 0.007*"ganadero" + 0.006*"trabajar"'), (1, '0.033*"ser" + 0.017*"haber" + 0.011*"parir" + 0.011*"\n\n" + 0.007*"hacer" + 0.007*"tener" + 0.007*"comer" + 0.007*"poder" + 0.006*"  " + 0.005*"madrid"'), (2, '0.020*"ser" + 0.009*"parir" + 0.009*"\n\n" + 0.009*"financiero" + 0.008*"haber" + 0.006*"do" + 0.006*"pasar" + 0.006*"athletic" + 0.006*"regulador" + 0.006*"entidad"'), (3, '0.021*"ser" + 0.012*"parir" + 0.008*"españa" + 0.008*"comer" + 0.008*"historia" + 0.007*"segundar" + 0.007*"agustina" + 0.006*"haber" + 0.006*"niño" + 0.006*"aragón"'), (4, '0.021*"ser" + 0.015*"haber" + 0.014*"parir" + 0.013*"\n\n" + 0.011*"comer" + 0.008*"euro" + 0.006*"entrar" + 0.005*"marcar" + 0.005*"producto" + 0.004*"poder"'), (5, '0.025*"ser" + 0.017*"\n\n" + 0.016*"haber" + 0.009*"parir" + 0.009*"tener" + 0.007*"hacer" + 0.007*"comer" + 0.006*"partir" + 0.006*"triple" 

pyLDAvis nos permite obtener los datos obtenidos en el ançalisis LDA de forma gráfica. Una vez realizado el análisis, habilitamos que los resultados se puedan visualizar en el propio notebook con pyLDAvis.enable_notebook(), y preparamos los datos. Los atributos de la función prepare son el resultado del análisis (ldamodel), el corpus y el diccionario.
Por último, los datos preparados se pueden visualizar de forma gráfica con la función display. Es una visualización interactiva, cada vez que se pulsa uno de los tópicos, se representan las palabras que lo componen

In [11]:
pyLDAvis.enable_notebook()

data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)


/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [12]:
pyLDAvis.display(data)